In [ ]:
!pip install transformers torch accelerate torchtune huggingface_hub json

Hugging Face Login


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
from huggingface_hub import login

login()

Model information

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Test Prompts

In [ ]:
# Input prompt
input_text = "Please generate a text based riddle."
inputs = tokenizer(input_text, return_tensors="pt")

# Generate text
outputs = model.generate(
    inputs["input_ids"].to("cuda"),  # Send input to GPU if available
    max_length=1000,  # Maximum length of the output
    num_return_sequences=1,  # Number of responses
    temperature=0.8,  # Adjust creativity level
    top_p=0.9,  # Nucleus sampling
)

# Decode and print the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
output = None # insert prompt to LLM here

Location nodes and map

In [11]:
class Location:
  def __init__(self, block = False):
    self.blocked = block
    # room connections
    self.north = None
    self.east = None
    self.west = None
    self.south = None
    # list of items - there's one case where there can be more than one item but its useful
    self.items = []
    # description of the room
    self.desc = None
    # bool as to whether a riddle can occur in the room
    self.riddle = False
    # message that the parser reads if the way is blocked.
    self.blocked_msg = None

In [35]:
 Basicest map
start = Location()
hallway = Location()
candleRoom = Location()
bugRoom = Location(True)
escapeRoom = Location()

# Joining up the locations
start.east = hallway

hallway.east = candleRoom
hallway.west = start
hallway.south = bugRoom

candleRoom.west = hallway

bugRoom.north = hallway
bugRoom.south = escapeRoom

escapeRoom.north = bugRoom

# Set items in the rooms
candleRoom.items.append("candle")
hallway.items.append("rocks")

# Establishes the rooms the riddles will occur in
start.riddle = True
bugRoom.riddle = True
escapeRoom.riddle = True

# Set descriptions for the room
start.desc = """The room you woke up in. The air is moist and the smell of mildew fills your lungs. There is an opening to the east."""

hallway.desc = """A room that connects to three others.
It's dim but you can faintly see a light from the room to the west.
The room to the south is loud. It sounds like something is alive down there.
The room to the east is where you woke up."""

candleRoom.desc = """A surprisingly bright room for the tomb. Its lined with many fires, and the walls seem gilded with gold. You're unfortunately unable to collect any.
The room to the west is the hallway."""

bugRoom.desc = """The sound grows louder and louder until you realize what it is. Bugs. The floor is covered in them and they all seem aggressive.
The room to the north is the hallway.
To the south seems to be your escape"""

escapeRoom.desc = """You can just barely see the daylight! You're nearly freed.
'escape' to free yourself!
"""

# Set block messages to the room
bugRoom.blocked_msg = "The room is teeming with bugs you're unable to take so much as a step inside without them crawling on you, biting your legs up."
escapeRoom.blocked_msg = "You cannot leave. The Sphinx won't let you"

Parser class

In [36]:
class Parser:
  def __init__(self, debug = False):
    self.lives = 3
    self.location = start
    self.inventory = []
    self.hint = None
    self.question = None
    self.answer = None
    self.explanaton = None
    self.level = 0
    self.victory = False
    self.debug = debug

  def ask(self):
    # promtpts the LLM to output a new riddle in the form of a json file
    # that includes the riddle, a hint and the answer
    #
    # inputs : None
    # output : The question from the model's response.
    if (self.question != None):
      return self.question
    else:
      if self.level == 3:
        difficult = "very hard"
      elif self.level == 2:
        difficult = "hard"
      else:
        difficult = "easy"

      one_shot = "I follow you everywhere yet never make a sound. I grow long when the day grows short, and disappear when the light is gone. What am I?"

      # Input prompt
      input_text = """Generate a {difficult} text based riddle.
      You may not include the following words in the answer: "help", "quit", "new", "hint"
      Respond in json format providing a riddle, three progressively revealing hints, an answer and an explanation.
      The keys should strictly be named "riddle", "hints", "answer", and "explanation" respectively.
      Use the following riddle as an example {one_shot}"""

      inputs = tokenizer(input_text, return_tensors="pt")

      # Generate text
      outputs = model.generate(
          inputs["input_ids"].to("cuda"),  # Send input to GPU if available
          max_length=1000,  # Maximum length of the output
          num_return_sequences=1,  # Number of responses
          temperature=0.8,  # Adjust creativity level
          top_p=0.9,  # Nucleus sampling
      )

      # Decode and print the output
      generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
      if generated_text:
        jsonOutput = json.loads(generated_text)
      else:
        jsonOutput = generated_text
      output = None # insert prompt to LLM here

      try:
        self.question = jsonOutput["riddle"]
        self.hint = jsonOutput["hints"]
        self.answer = jsonOutput["answer"]
        self.explanation = jsonOutput["explanation"]
      except Exception as e:
        self.ask() # start over if the model didn't properly format the json file.

      return self.question

  def answer(self, response):
    # answer - compares the response given to the answer the model provided.
    #
    # inputs: user response
    # returns: True or False

    response_split = response.split(" ")
    eval = 0.0

    for e in len(response_split):
      if(response_split[e] in self.answer):
        eval += (1 / len(self.answer.split(" "))) # tries to match words from the response to words in
      else:
        eval -= 0.1 # punishes answers that guess everything

    return True if eval > 0.5 else False # Dummy variable for now

  def prologue(self):
    # Prologue -
    # returns nothing but easier to navigate than multiple strings
    print("""Welcome to the game. Placeholder text for the openning.
    More story stuff to come.
    help for list of commands.""") # more to come than this but its a placeholder

  def game(self):
    # Game - Provides the game loop and main functionality of the game.
    # The game ends when the player quits, runs out of lives or they
    # correctly respond to three riddles.

    playing = True

    self.prologue()

    while(playing and self.lives > 0 and not self.victory): # GAMELOOP

      # Room message.
      print(self.location.desc)
      if(self.location.blocked):
        print("You aren't be able to go any further south at this time.")
      if (self.location.items): # If there are items they're listed
        print("You see something useful: ")
        for item in self.location.items:
          print("\t a " + item)

      # Takes user response and splits it before going through commands
      response = input("")
      response = response.lower()
      responseList = response.split(" ")

      # the big switch case controlling our game.
      if(responseList[0] == "q" or responseList[0] == "quit"):
        playing = False # User ends the game on their own, it ends here

      # Directional controls for the location system.
      elif(responseList[0] == "n" or responseList[0] == "north"):
          if self.location.north:
            if not self.location.riddle or self.debug:
                self.location = self.location.north
            else:
              print("The sphinx has you trapped in this room until you can answer her riddle!")
      elif(responseList[0] == "e" or responseList[0] == "east"):
          if self.location.east:
            if not self.location.riddle or self.debug:
              self.location = self.location.east
            else:
              print("The sphinx has you trapped in this room until you can answer her riddle!")
      elif(responseList[0] == "w" or responseList[0] == "west"):
          if self.location.west:
            if not self.location.riddle or self.debug:
              self.location = self.location.west
            else:
              print("The sphinx has you trapped in this room until you can answer her riddle!")
      elif(responseList[0] == "s" or responseList[0] == "south"):
        if self.location.south:
          if not self.location.blocked: # Progress will only ever go south, so we only need to check if its blocked here
              if not self.location.riddle or self.debug: # checks if in debug mode or if you've finished the riddle
               self.location = self.location.south
              else:
                print("The sphinx has you trapped in this room until you can answer her riddle!")
          else:
            print(self.location.blocked_msg)

      # give up command - going to phase out as we'll move the riddles to the ask function
      elif(responseList[0] == "give" and responseList[1] == "up"):
         print("You decide to move on to the next question")
         self.lives -= 1
         self.hint = None
         self.question = None
         self.answer = None

      # look around - redundant but it lets you see everything again
      elif(responseList[0] == "look"):
        print(self.location.desc)
        if(self.location.blocked):
          print("You are won't be able to go any further south at this time.")
        if (self.location.items): # If there are items they're listed
          print("You see something useful: ")
          for item in self.location.items:
            print("\t a " + item)

      # get - it allows players to
      elif(responseList[0] == "get"):
        if self.location.items:
          self.inventory = self.inventory + self.location.items # conjoins the lists
          self.location.items = [] # empties out the list of items
        else:
          print("Have you gone mad already? There's nothing there.")

      # light - our main text based puzzle - players light up their candle to unblock the bug room
      elif(responseList[0] == "light" or "fire" in responseList):
        if "candle" or "wood" in self.inventory:
          if self.location == bugRoom:
            if "rocks" in self.inventory:
              print("You start a fire that scares the bugs away. You're now free to continue through this room")
              bugRoom.blocked = False
              bugRoom.desc = "Updated message that no longer has bugs"
          else:
            print("There's no reason to do that here")
        else:
          print("Despite your attempts to start a fire with your mind you have nothing to set ablaze")

      # help gives the controls
      elif(responseList[0] == "help" or responseList[0] == "h"):
        print("Controls :")
        print("\t" + "q/quit - exits out of the game")

        print("\t" +"Directions: ")
        print("\t" + "\t" + "n/north - moves to the room north of this one")
        print("\t" + "\t" + "e/east - moves to the room east of this one")
        print("\t" + "\t" + "s/south - moves to the room south of this one")
        print("\t" + "\t" + "w/west - moves to the room west of this one")

        print("\t" +"look - describes the room and any items in it.")
        print("\t" +"inventory - shows you the items you've obtained")
        print("\t" +"get - adds all items in a room to your inventory.")
        print("\t" +"There are more commands related to any items you might find! Try things out!")

      # else - the player misinput
      else:
        print("Not a registered command. Please type 'help' for a list of all controls.")


      # else:

      #   eval = self.answer(response)

      # if(eval): #Placeholder for succeeding the riddle. We'll add an evaluation method later.
      #   # Tells player they lost, decrements lives, and resets hint, question and answer variables
      #   print("Congrats that was the correct response")
      #   self.level += 1
      #   self.hint = None
      #   self.question = None
      #   self.answer = None

      # elif (eval): #Placeholder for succeeding the riddle. We'll add an evaluation method later
      #   print("Incorrect response. Please try again")
      #   self.life -= 1


    # The gameloop has ended.
    if self.victory:
      print("Congratulations! You're free from the sphinx's game and did not meet a gruesome fate.")
    else:
      if self.lives >= 0:
        print("Your adventure ends here. Your heart comes to a sudden stop.")
      print("The tomb seals shut, dooming you to spend the rest of your short life within this miserable pit.")

    return


In [37]:
pharohs_game = Parser(True)
pharohs_game.game()

Welcome to the game. Placeholder text for the openning.
    More story stuff to come.
    help for list of commands.
The room you woke up in. The air is moist and the smell of mildew fills your lungs. There is an opening to the east.
e
A room that connects to three others.
It's dim but you can faintly see a light from the room to the west.
The room to the south is loud. It sounds like something is alive down there.
The room to the east is where you woke up.
You see something useful: 
	 a rocks
e
A surprisingly bright room for the tomb. Its lined with many fires, and the walls seem gilded with gold. You're unfortunately unable to collect any.
The room to the west is the hallway.
You see something useful: 
	 a candle
get
A surprisingly bright room for the tomb. Its lined with many fires, and the walls seem gilded with gold. You're unfortunately unable to collect any.
The room to the west is the hallway.
w
A room that connects to three others.
It's dim but you can faintly see a light from